# Sieci Neuronowe, PWR 2023
## Klasyfikacja roślin na podstawie zdjęć za pomocą sieci neuronowych
Autorzy:
- Dominik Ćwikowski 248914
- Paweł Pelar 243480  

Repozytorium:  
- https://github.com/F3mte/P-Sieci-Neuronowe

Zbiór danych:  
- https://www.kaggle.com/datasets/marquis03/plants-classification

In [1]:
# Import bibliotek i funkcji
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

2023-12-11 19:38:17.507314: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Określ ścieżki do folderów z plikami 
test_dir = "test"
train_dir = "train"
val_dir = "val"

In [3]:
# Generator dla danych treningowych
train_datagen = ImageDataGenerator(rescale=1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   rotation_range=180,
                                   fill_mode='reflect')
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    class_mode='categorical',
                                                    target_size=(224, 224))

Found 21000 images belonging to 30 classes.


In [4]:
# Generator dla danych testowych
test_datagen = ImageDataGenerator(rescale=1./255,)
test_generator = test_datagen.flow_from_directory(directory=test_dir,
                                                  class_mode='categorical',
                                                  target_size=(224, 224))

Found 6000 images belonging to 30 classes.


In [5]:
# Generator dla danych walidacyjnych
validation_datagen = ImageDataGenerator(rescale=1./255,)
validation_generator = validation_datagen.flow_from_directory(directory=val_dir,
                                                              class_mode = 'categorical',
                                                              target_size=(224, 224))

Found 3000 images belonging to 30 classes.


In [ ]:
x_train = []
c = 0
for feature, label in train_generator:
    x_train.append(np.array(feature))
    c += 1
    if c == 1:
        break

x_train = np.array(x_train)
print(x_train.shape)
x_train = np.reshape(x_train, (32, 224, 224, 3))
print(x_train.shape)

fig, ax = plt.subplots(4, 4, figsize=(15, 15))
for i in range(0, 16):
    ax[int(i / 4), (i % 4)].imshow(x_train[i])

# Proponowana architektura

Input 224x224x3  

Conv2D 64, 3x3, relu  
MaxPooling 2x2  

Conv2D 64, 3x3, relu  
MaxPooling 2x2  

Conv2D 64, 3x3, relu  
MaxPooling 2x2  

Flatten  
Dropout (0.2 - 0.5)  

Dense 4096, relu  
Dropout (0.2 - 0.5) 

Dense 30, softmax  

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Definiowanie modelu z podaną architekturą
model = Sequential([
    # Warstwa wejściowa: Input 224x224x3 jest domyślnie określona przez target_size w generatorze
    # Pierwsza warstwa konwolucyjna
    Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    # Druga warstwa konwolucyjna
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Trzecia warstwa konwolucyjna
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Spłaszczanie danych do jednowymiarowej tablicy
    Flatten(),

    # Dropout (0.2 - 0.5), użyjemy średniej wartości 0.35
    Dropout(0.35),

    # Pierwsza warstwa gęsta
    Dense(4096, activation='relu'),
    Dropout(0.35),

    # Warstwa wyjściowa
    Dense(30, activation='softmax')
])

# Podsumowanie modelu
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 64)        0

# Trenowanie modelu

In [7]:
# Kompilacja modelu
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Callback do zapisywania najlepszego modelu
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

# Trenowanie modelu
history = model.fit(
    train_generator,
    epochs=10, 
    validation_data=validation_generator,
    callbacks=[checkpoint]
)

# Zapisanie całego modelu
model.save('final_model.h5')

Epoch 1/10


KeyboardInterrupt: 

# Testowanie modelu

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Wczytanie modelu
model = load_model('final_model.h5')

# Testowanie modelu na danych testowych
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Generowanie przewidywań na danych testowych
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Etykiety rzeczywiste
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())  

# Wyświetlanie raportu klasyfikacji
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Wyświetlanie macierzy pomyłek
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)
